# Pre-trained embeddings for Text

In [ ]:
import gzip
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
glove_path = '../data/embeddings/glove.6B.50d.txt.gz'

In [ ]:
with gzip.open(glove_path, 'r') as fin:
    line = fin.readline().decode('utf-8')

In [ ]:
line

In [ ]:
def parse_line(line):
    values = line.decode('utf-8').strip().split()
    word = values[0]
    vector = np.asarray(values[1:], dtype='float32')
    return word, vector

In [ ]:
embeddings = {}
word_index = {}

In [ ]:
word_inverted_index = []

In [ ]:
with gzip.open(glove_path, 'r') as fin:
    for idx, line in enumerate(fin):
        word, vector = parse_line(line) # parse a line
        
        embeddings[word] = vector  # add word vector
        word_index[word] = idx  # add idx
        word_inverted_index.append(word)  # append word

In [ ]:
word_index['good']

In [ ]:
word_inverted_index[219]

In [ ]:
embeddings['good']

In [ ]:
embedding_size = len(embeddings['good'])
embedding_size

In [ ]:
plt.plot(embeddings['good']);

In [ ]:
plt.subplot(211)
plt.plot(embeddings['two'])
plt.plot(embeddings['three'])
plt.plot(embeddings['four'])
plt.title("A few numbers")
plt.ylim(-2, 5)

plt.subplot(212)
plt.plot(embeddings['cat'])
plt.plot(embeddings['dog'])
plt.plot(embeddings['rabbit'])
plt.title("A few animals")
plt.ylim(-2, 5)

plt.tight_layout()

In [ ]:
vocabulary_size = len(embeddings)
vocabulary_size

## Loading pre-trained embeddings in Keras

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding

In [ ]:
embedding_weights = np.zeros((vocabulary_size,
                              embedding_size))

In [ ]:
for word, index in word_index.items():
    embedding_weights[index, :] = embeddings[word]

In [ ]:
emb_layer = Embedding(input_dim=vocabulary_size,
                      output_dim=embedding_size,
                      weights=[embedding_weights],
                      mask_zero=False,
                      trainable=False)

In [ ]:
word_inverted_index[0]

In [ ]:
model = Sequential()
model.add(emb_layer)

In [ ]:
embeddings['cat'] 

In [ ]:
cat_index = word_index['cat']

In [ ]:
cat_index

In [ ]:
model.predict([[cat_index]])

## Gensim

In [ ]:
import gensim

In [ ]:
from gensim.scripts.glove2word2vec import glove2word2vec

In [ ]:
glove_path = '../data/embeddings/glove.6B.50d.txt.gz'
glove_w2v_path = '../data/embeddings/glove.6B.50d.txt.vec'

In [ ]:
glove2word2vec(glove_path, glove_w2v_path)

In [ ]:
from gensim.models import KeyedVectors

In [ ]:
glove_model = KeyedVectors.load_word2vec_format(
    glove_w2v_path, binary=False)

In [ ]:
glove_model.most_similar(positive=['good'], topn=5)

In [ ]:
glove_model.most_similar(positive=['two'], topn=5)

In [ ]:
glove_model.most_similar(positive=['king', 'woman'],
                         negative=['man'], topn=3)

## Visualization

In [ ]:
import os

In [ ]:
model_dir = '/tmp/tensorboard/'

In [ ]:
n_viz = 4000

In [ ]:
emb_layer_viz = Embedding(n_viz,
                          embedding_size,
                          weights=[embedding_weights[:n_viz]],
                          mask_zero=False,
                          trainable=False)

In [ ]:
model = Sequential([emb_layer_viz])

In [ ]:
word_embeddings = emb_layer_viz.weights[0]

In [ ]:
word_embeddings

In [ ]:
import tensorflow as tf

In [ ]:
checkpoint = tf.train.Checkpoint(model=model)
checkpoint.save(os.path.join(model_dir, 'model.ckpt'))

In [ ]:
os.listdir(model_dir)

In [ ]:
fname = os.path.join(model_dir, 'metadata.tsv')

with open(fname, 'w', encoding="utf-8") as fout:
    for index in range(0, n_viz):
        word = word_inverted_index[index]
        fout.write(word + '\n')

In [ ]:
config_string = """
embeddings {
  tensor_name: "model/layer_with_weights-0/embeddings/.ATTRIBUTES/VARIABLE_VALUE"
  metadata_path: "metadata.tsv"
}
"""

In [ ]:
fname = os.path.join(model_dir, 'projector_config.pbtxt')

with open(fname, 'w', encoding="utf-8") as fout:
    fout.write(config_string)

Now open tensorboard and play with the projector.